In [ ]:
# default_exp main

# Main

> The main FastAPI app.

In [ ]:
#export
from vulcan_rwd_ig.core import *
from fastapi import FastAPI, Header, HTTPException
from fastapi.staticfiles import StaticFiles
from pydantic import BaseModel
from typing import List, Dict
from datetime import datetime
from ast import literal_eval
from pathlib import Path
import os, json

In [ ]:
#export
def get_valid_api_keys():
    "Return a list of keys that can be used to authorise users of this API"
    keys = os.environ.get('X_API_KEYS')
    assert keys is not None, 'Environment variable X_API_KEYS not set'
    return literal_eval(keys)

In [ ]:
#export
def api_key_check(x_api_key):
    if x_api_key not in get_valid_api_keys():
        raise HTTPException(status_code=401, detail='Invalid x-api-key')

In [ ]:
#export
class CohortBuilderSearchPayload(BaseModel):
    fhir_base: str
    age: int
#     diagnosis_label: str
    diagnosis_icd10: str
#     diagnosis_icd10_label: str
    medication_rxnorm: str
#     medication_rxnorm_label: str

In [ ]:
#export
class CohortBuilder:
    def __init__(self, fhir_base):
        self.client = FhirClient(fhir_base)
        
    def search(self, age, diagnosis_icd10, medication_rxnorm):
        # TODO: use discharge date to calculate birthdate from age - for now we'll hard code
        discharge_year = 2014
        birth_year = discharge_year - age
        patient_resources = self.client.get_all_resources('Patient', {
            'birthdate': f'le{birth_year}-09-01', # 1996
            'gender': 'male,female'
        })
        encounter_resources = self.client.get_all_resources('Encounter', {
                'reason-code': diagnosis_icd10.replace(' ', ''), # TODO: not using below for now 'reason-code:below': 'I20,I21,I22,I23,I24,I25',
                'date': [f'ge{discharge_year}-09-01', f'le{discharge_year + 1}-09-30'], # TODO: Not sure this is filtering as we want ...
                'status': 'finished',
        #         'dischargeDisposition:not':'exp' # TODO: Do this client side for now
        })
        medication_rxnorm = medication_rxnorm.replace(' ', '')
        code = ','.join([f'http://www.nlm.nih.gov/research/umls/rxnorm|{n}' for n in medication_rxnorm.split(',')])
        print('rxnorm code', code)
        medication_administration_resources = self.client.get_all_resources('MedicationAdministration', {
                'status': 'completed',
        #         'effective-time': 'ge[Encounter-Start-Date]', # TODO: don't think this is possible via FHIR query
                'code': code})
        patient_ids = intersection_patient_ids(patient_resources, encounter_resources, medication_administration_resources)
        return {
            'fhir_base': self.client,
            'echo': (age, diagnosis_icd10, medication_rxnorm),
            'patient_ids': patient_ids
        }

In [ ]:
#export
def create_app():
    app = FastAPI()
    app.mount('/static', StaticFiles(directory='static'), name='static')
    
    @app.get('/api-key-check')
    def _api_key_check(x_api_key: str = Header(None)):
        "Validates the `x-api-key` header parameter"
        api_key_check(x_api_key)
        return {'x-api-key': x_api_key, 'result': 'OK'}
    
    @app.options('/cohort_building_search/')
    def _translate_options(x_api_key: str = Header(None)):
        "Validate endpoint before invoking the actual POST method"
#         api_key_check(x_api_key)

    @app.post('/cohort_building_search/') # , response_model=Dict[str, str]
    def _cohort_building_search(payload: CohortBuilderSearchPayload, x_api_key: str=Header(None)):
        "TODO"
#         api_key_check(x_api_key)
        print('x_api_key', x_api_key)
        print(payload)
        cohort_builder = CohortBuilder(payload.fhir_base)
        return cohort_builder.search(payload.age, payload.diagnosis_icd10, payload.medication_rxnorm)
        
    return app

In [ ]:
import requests
data = {
    'fhir_base': 'https://fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io', 
    'age': 18, 
    'diagnosis_icd10': 'I20, I21, I22, I23, I24, I25', 
    'medication_rxnorm': '1116632, 613391, 32968, 687667, 153658'}
headers = {'x-api-key': 'who-cares'}
response = requests.post('http://127.0.0.1:8000/cohort_building_search', json.dumps(data), headers=headers)
response.text

'{"fhir_base":{"api_base":"https://fhir.ggyxlz8lbozu.workload-prod-fhiraas.isccloud.io","use_local_cache":true,"request_headers":{},"default_params":{}},"echo":[18,"I20, I21, I22, I23, I24, I25","1116632,613391,32968,687667,153658"],"patient_ids":["Patient/a3f4f0ffc5c2fb1a4708452a485d1442","Patient/1bbc2bc53ed277ac09507e6893743410","Patient/00d7dde9ae58163184c3836f01deff61","Patient/6863ac983b0b55455da78f1fdd1288ff","Patient/ff7c22942a1e16167f1b9c44f12aae05","Patient/897dfe86f0b710793927d8034e568ee4","Patient/f7f2e775f7ae3f7a095146cb4deaa497","Patient/51ff4d27ccf78c1d2ff6438175b541c7","Patient/e4c9f85f8b2b9a85b32f7d9a67ea1046","Patient/ceba63b6dcbda783668cf3efeea1d3dd"]}'

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script('10_main.ipynb')

Converted 10_main.ipynb.
